In [2]:
import pandas as pd
from utils.subgraph import KlerosBoardSubgraph
from plotly import graph_objects as go


In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
kb_eth = KlerosBoardSubgraph(network='mainnet')
kb_gno = KlerosBoardSubgraph(network='gnosis')

In [5]:
stakes_eth = kb_eth.getAllStakeSets()
stakes_gno = kb_gno.getAllStakeSets()

In [6]:
df_stakes_eth = pd.DataFrame(stakes_eth)
df_stakes_gno = pd.DataFrame(stakes_gno)

In [19]:
# PNK staked history
from datetime import datetime
from utils.utils import getTimeSeriePNKStaked


pnk_staked_eth = getTimeSeriePNKStaked(df_stakes_eth)
pnk_staked_gno = getTimeSeriePNKStaked(df_stakes_gno)



total_supply = 776626704
pnk_staked_eth['total_supply'] = 776626704
pnk_staked_eth.loc[pnk_staked_eth.index < datetime(year=2024,month=1,day=20), 'total_supply'] -= 12000000  # mint kip66
pnk_staked_eth.loc[pnk_staked_eth.index < datetime(year=2019,month=12,day=30), 'total_supply'] -= 200000000  # mint second token sale

pnk_staked_eth['percentage'] = pnk_staked_eth['total_staked'] / pnk_staked_eth['total_supply']
pnk_staked_gno['percentage'] = pnk_staked_gno['total_staked'] / pnk_staked_eth['total_supply']


/home/koki/Documentos/Personales/repos/klerosboard_stats/utils/utils.py:53: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.

/home/koki/Documentos/Personales/repos/klerosboard_stats/utils/utils.py:53: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.



In [37]:
df = pd.merge_asof(
    left=pnk_staked_eth, right=pnk_staked_gno,
    left_index=True, right_index=True,
    suffixes=['','gno_'],
    )
print(df.tail())


fig = go.Figure(data=[
    go.Bar(x=pnk_staked_eth.index, y=pnk_staked_eth['percentage'] * 100, name="Ethereum", offsetgroup='a'),
    go.Bar(x=pnk_staked_gno.index, y=pnk_staked_gno['percentage'] * 100, name="Gnosis", offsetgroup='a', base=pnk_staked_eth.loc[base, 'percentage'] * 100),
    ]
    )
fig.update_layout(
     xaxis=dict(showgrid=False), 
     yaxis=dict(showgrid=False, range=[0, 30]),
     title=dict(text="% PNK staked over Total Supply")
)
fig.show()



            total_staked  total_supply  percentage  total_stakedgno_  \
2023-10-31  1.788536e+08     764626704    0.233910      1.580733e+07   
2023-11-30  1.874996e+08     764626704    0.245217      2.138479e+07   
2023-12-31  1.859819e+08     764626704    0.243232      3.290337e+07   
2024-01-31  1.976793e+08     776626704    0.254536      2.377725e+07   
2024-02-29  2.011647e+08     776626704    0.259024      3.263565e+07   

            percentagegno_  
2023-10-31        0.020673  
2023-11-30        0.027968  
2023-12-31        0.043032  
2024-01-31        0.030616  
2024-02-29        0.042022  


NameError: name 'base' is not defined

In [ ]:
from utils.utils import getTimeSerieActiveJurors

active_jurors_eth: pd.DataFrame = getTimeSerieActiveJurors(df_stakes_eth)
active_jurors_gno: pd.DataFrame = getTimeSerieActiveJurors(df_stakes_gno)


KeyError: -1

In [ ]:
print(f'Active Jurors:\nMainnet:{active_jurors_eth.iloc[-1].values[0]}\nGnosis:{active_jurors_gno.iloc[-1].values[0]}')

Active Jurors:
Mainnet:778
Gnosis:119


In [ ]:


fig = go.Figure(data=[
    go.Scatter(x=active_jurors_eth.index, y=active_jurors_eth['active_jurors'], mode="lines", name="Ethereum"),
    go.Scatter(x=active_jurors_gno.index, y=active_jurors_gno['active_jurors'], mode="lines", name="Gnosis"),
    ]
    )
fig.update_layout(
     xaxis=dict(showgrid=False), 
     yaxis=dict(showgrid=False),
     title=dict(text="Daily Active Jurors")
)
fig.show()


In [ ]:
growth_eth = active_jurors_eth.diff()
growth_gno = active_jurors_gno.diff()

growth_eth = growth_eth.resample('M').sum()
growth_gno = growth_gno.resample('M').sum()

fig = go.Figure(data=[
    go.Bar(x=growth_eth.index, y=growth_eth['active_jurors'], name="Ethereum"),
    go.Bar(x=growth_gno.index, y=growth_gno['active_jurors'], name="Gnosis"),
    ]
    )
fig.update_layout(
     xaxis=dict(showgrid=False), 
     yaxis=dict(showgrid=False),
     title=dict(text="Monthly Active Jurors Growth")
)
fig.add_hline(y=0)
fig.show()


/tmp/ipykernel_138916/2645704541.py:4: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.

/tmp/ipykernel_138916/2645704541.py:5: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.



In [ ]:
# Staked amount of PNKs over history


/tmp/ipykernel_138916/1766111282.py:1: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.



,active_jurors
2019-03-31,41.0
2019-04-30,11.0
2019-05-31,10.0
2019-06-30,18.0
2019-07-31,-7.0
...,...
2023-11-30,-1.0
2023-12-31,-4.0
2024-01-31,6.0
2024-02-29,-9.0


In [ ]:
txs_eth = kb_eth.getAllTransactions()
txs_gno = kb_gno.getAllTransactions()


In [ ]:
daily_txs_eth = txs_eth.resample('M', on='timestamp').count()
daily_txs_gno = txs_gno.resample('M', on='timestamp').count()


fig = go.Figure(data=[
    go.Bar(x=daily_txs_eth.index, y=daily_txs_eth['tx'], name="Ethereum"),
    go.Bar(x=daily_txs_gno.index, y=daily_txs_gno['tx'], name="Gnosis"),
    ]
    )
fig.update_layout(
     xaxis=dict(showgrid=False), 
     yaxis=dict(showgrid=False),
     title=dict(text="Monthly txs in Kleros Courts")
)
fig.add_hline(y=0)
fig.show()

/tmp/ipykernel_138916/1509046466.py:1: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.

/tmp/ipykernel_138916/1509046466.py:2: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.



In [ ]:
token_txs_eth = kb_eth.getAllTransfers()
token_txs_gno = kb_gno.getAllTransfers()

In [ ]:
transfers_eth = pd.DataFrame(token_txs_eth)
transfers_gno = pd.DataFrame(token_txs_gno)

transfers_eth['timestamp'] = pd.to_datetime(transfers_eth.timestamp, unit='s')
transfers_gno['timestamp'] = pd.to_datetime(transfers_gno.timestamp, unit='s')
transfers_eth.sort_values('timestamp', inplace=True)
transfers_gno.sort_values('timestamp', inplace=True)


In [ ]:
# Get ETH price
from utils.oracles import CoinGecko
from datetime import datetime, timedelta

days_before = (datetime.now() - transfers_eth.timestamp.min()).days
eth_price = CoinGecko().getETHhistoricPrice(days_before)
eth_price = pd.DataFrame(eth_price, columns=['timestamp', 'price'])
eth_price['timestamp'] = pd.to_datetime(eth_price['timestamp'], unit='ms')


In [ ]:
transfers_eth.columns


Index(['id', 'ETHAmount', 'tokenAmount', 'blockNumber', 'timestamp'], dtype='object')

In [ ]:

transfers_eth_price = pd.merge_asof(
    left=transfers_eth, right=eth_price,
    on='timestamp', direction='forward', tolerance=timedelta(hours=23)
    )
transfers_eth_price['ETHAmount_usd'] = transfers_eth_price['ETHAmount'] * transfers_eth_price['price']

In [ ]:

transfers_eth_resampled = transfers_eth_price.resample('M', on='timestamp')['ETHAmount_usd'].sum()
transfers_gno_resampled = transfers_gno.resample('M', on='timestamp')['ETHAmount'].sum()

/tmp/ipykernel_138916/2002161228.py:1: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.

/tmp/ipykernel_138916/2002161228.py:2: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.



In [ ]:
fig = go.Figure(data=[
    go.Bar(x=transfers_eth_resampled.index, y=transfers_eth_resampled, name="Ethereum"),
    go.Bar(x=transfers_gno_resampled.index, y=transfers_gno_resampled, name="Gnosis"),
    ]
    )
fig.update_layout(
     xaxis=dict(showgrid=False), 
     yaxis=dict(showgrid=False),
     title=dict(text="Monthly Fees payed to jurors [USD]")
)
fig.add_hline(y=0)
fig.show()


fig = go.Figure(data=[
    go.Scatter(x=transfers_eth_resampled.index, y=transfers_eth_resampled.cumsum(), name="Ethereum", mode='lines'),
    go.Scatter(x=transfers_gno_resampled.index, y=transfers_gno_resampled.cumsum(), name="Gnosis", mode='lines'),
    ]
    )
fig.update_layout(
     xaxis=dict(showgrid=False), 
     yaxis=dict(showgrid=False),
     title=dict(text="Cumulative Fees payed to jurors [USD]")
)
fig.add_hline(y=0)
fig.show()

In [ ]:
jurors_eth = pd.DataFrame(kb_eth.getAllJurors())
jurors_gno = pd.DataFrame(kb_gno.getAllJurors())

In [ ]:
jurors_eth = jurors_eth[jurors_eth.ethRewards > 0][['totalStaked', 'id', 'ethRewards', 'numberOfDisputesAsJuror']]
jurors_gno = jurors_gno[jurors_gno.ethRewards > 0][['totalStaked', 'id', 'ethRewards', 'numberOfDisputesAsJuror']]


In [ ]:
from utils.utils import gini


gini_eth = gini(jurors_eth['ethRewards'], jurors_eth['totalStaked'])
gini_gno = gini(jurors_gno['ethRewards'], jurors_gno['totalStaked'])

print(gini_eth, gini_gno)

0.5978305812265594 0.36928317393004656
